In [1]:
### This file convert the .json file to the .xlsx file

In [1]:
import json
import os
import pandas as pd

from openpyxl import Workbook

In [2]:
file1 = r'C:\Users\Administrator\Desktop\PPG_dataset\json\seven_grade.json'
file2 = r'C:\Users\Administrator\Desktop\PPG_dataset\json\eight_grade.json'

In [3]:
def Convert2xlsx(filePath, fileName):
    wb = Workbook()
    ws = wb.active
    ws.title = "遂昌"
    with open(filePath, 'r', encoding='utf-8-sig') as file:
        datas = json.load(file)['data']

    columns = ["user_id", 'user_name', 'gender']
    for idx, data in enumerate(datas):
        # 写入数据
        user_id = data["userId"]
        user_name = data["userName"]
        gender = data["gender"]
        dimensions = data["dimensions"]
        selected_option_content_list = []
        for dimension in dimensions:
            question_title = dimension["question"]["title"]
            selected_option_content = dimension["selectedOption"]["content"]

            if idx == 0:
                columns.append(question_title)

            if selected_option_content == '是':
                selected_option_content_list.append(1)
            elif selected_option_content == '否':
                selected_option_content_list.append(0)
            else:
                raise
        if idx == 0:
            ws.append(columns)
        ws.append([user_id, user_name, gender] + selected_option_content_list)

    # 保存为 xlsx 文件
    wb.save(f"{fileName}.xlsx")
    print(f"数据已成功保存到: {fileName}.xlsx 文件中!")


outdir = "outdir"
if not os.path.exists(outdir):
    os.makedirs(outdir)

Convert2xlsx(file1, f"./{outdir}/七年级")
Convert2xlsx(file2, f"./{outdir}/八年级")

数据已成功保存到: ./outdir/七年级.xlsx 文件中!
数据已成功保存到: ./outdir/八年级.xlsx 文件中!


In [33]:
seven_grade_df = pd.read_excel(f"./{outdir}/七年级.xlsx")
eight_grade_df = pd.read_excel(f"./{outdir}/八年级.xlsx")
seven_grade_df['age'] = len(seven_grade_df)*[13]
eight_grade_df['age'] = len(eight_grade_df)*[14]
combined_df = pd.concat([seven_grade_df, eight_grade_df], ignore_index=True)
combined_df['total'] = combined_df.iloc[:, 3:-1].sum(axis=1)
# 计算排除前3列后的其他列的和，并将其添加为一列 'total'

print(combined_df.shape)

(1011, 105)


In [22]:
seven_grade_df.columns

Index(['user_id', 'user_name', 'gender', '你晚上要睡觉时，是否总想着明天的功课? ',
       '老师向全班提问时，你是否会觉得是在问自己而感到不安? ', '你是否一听说“要考试”心里就紧张。 ',
       '你考试成绩不好时，心里是否感到很不快? ', '你学习成绩不好时，是否总是提心吊胆? ',
       '你考试时，想不起原先掌握的知识时，是否会感到紧张不安? ', '你考试后，在没有知道成绩之前，是否总是放心不下? ',
       ...
       '你以前是否说过谎话? ', '你是否一次也没有失约过? ', '你是否能保密别人不让说的事? ', '你是否总是很有礼貌? ',
       '老师或父母说的话，你是否都照办? ', '你是否发过怒? ', '你不喜欢的功课老师提前下课，你是否会感到特别高兴？ ',
       '你是否无论对谁都很亲热? ', '对不认识的人，你是否会都喜欢? ', 'age'],
      dtype='object', length=104)

In [35]:
df_filtered_by_total = combined_df[(combined_df['total'] >= 10) & (combined_df['total'] <= 90)]

print(df_filtered_by_total.shape)

(898, 105)


In [36]:
lying_columns = ['你以前是否说过谎话? ', '你是否发过怒? ']

In [37]:

final_filtered_df = df_filtered_by_total[(df_filtered_by_total[lying_columns] == 1).all(axis=1)]
print(final_filtered_df.shape)
print(f"After filter, the left available subject count is: {final_filtered_df.shape[0]}")

tmp_df = final_filtered_df[lying_columns]
print(tmp_df)

(571, 105)
After filter, the left available subject count is: 571
      你以前是否说过谎话?   你是否发过怒? 
1             1.0       1.0
2             1.0       1.0
3             1.0       1.0
4             1.0       1.0
6             1.0       1.0
...           ...       ...
1003          1.0       1.0
1005          1.0       1.0
1006          1.0       1.0
1008          1.0       1.0
1010          1.0       1.0

[571 rows x 2 columns]


In [38]:
depression_columns = ['你和大家在—起时，是否觉得自己是多余的人? ', '你是否经常会突然想哭? ', '你有时是否会觉得，还是死了好？ ', '你是否经常想从高的地方跳下来? ']
final_filtered_df['depression_sum'] = final_filtered_df[depression_columns].sum(axis=1)

print(final_filtered_df.shape)

(571, 106)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_164776\149341691.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_filtered_df['depression_sum'] = final_filtered_df[depression_columns].sum(axis=1)


In [39]:
# 添加 depression_label 列
final_filtered_df['depression_label'] = final_filtered_df['depression_sum'].apply(lambda x: 1 if x >= 4 else (0 if x == 0 else -1))

print(final_filtered_df.shape)

(571, 107)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_164776\539766930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_filtered_df['depression_label'] = final_filtered_df['depression_sum'].apply(lambda x: 1 if x >= 4 else (0 if x == 0 else -1))


## Tag the Anxies Label

In [40]:
anxiety_columns = ['你晚上要睡觉时，是否总想着明天的功课? ', '你是否一听说“要考试”心里就紧张。 ', '你学习成绩不好时，是否总是提心吊胆? ', '你考试时，想不起原先掌握的知识时，是否会感到紧张不安? ', \
                   '你考试后，在没有知道成绩之前，是否总是放心不下? ', '你是否一遇到考试，就担心会考坏? ', '你在没有完成任务之前，是否总担心完不成任务？ ', '你当着大家面朗读课文时，是否总是怕读错? ',
                   '你是否认为学校里得到的学习成绩总是不大可靠? ', '你是否认为你比别人更担心学习? ', '你是否做过考试考坏了的梦? ', '你是否做过学习成绩不好时，受到爸爸妈妈或老师训斥的梦? ', '你在工作或学习时，如果有人在注意你，你心里是否会紧张? ', \
                   '你受到老师批评时，心里是否总是不安? ', '你心里是否总觉得好像有什么事没有做好? ', '你是否总担心会发生什么意外的事？ ', '你被老师提问时，心里是否总是很紧张? ', '你是否经常咬指甲? ', '你是否经常感到呼吸困难? ', '你是否会经常急躁得坐立不安? ']

print(f"anxiety item count: {len(anxiety_columns)}")

anxiety item count: 20


In [41]:
for col in anxiety_columns:
    print(col in final_filtered_df.columns.values)
    

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [42]:
final_filtered_df['anxiety_sum'] = final_filtered_df[anxiety_columns].sum(axis=1)

# 添加 anxiety_label 列
final_filtered_df['anxiety_label'] = final_filtered_df['anxiety_sum'].apply( lambda x: 1 if x > 15 else (0 if x < 6 else -1))

print(final_filtered_df.shape)

(571, 109)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_164776\2206918076.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_filtered_df['anxiety_sum'] = final_filtered_df[anxiety_columns].sum(axis=1)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_164776\2206918076.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_filtered_df['anxiety_label'] = final_filtered_df['anxiety_sum'].apply( lambda x: 1 if x > 15 else (0 if x < 6 else -1))


In [43]:

final_filtered_df.to_excel(f"./{outdir}/MiddleSchool_Depression_Anxiety.xlsx", index=False)

anxiety_df = final_filtered_df[final_filtered_df['anxiety_label'].isin([0, 1])]
anxiety_df = anxiety_df[['user_id'] + anxiety_columns + ['anxiety_sum']]

In [44]:
stress_column = ['你是否一听说“要考试”心里就紧张。 ','你考试成绩不好时，心里是否感到很不快? ','你学习成绩不好时，是否总是提心吊胆? ','你考试时，想不起原先掌握的知识时，是否会感到紧张不安? ','你是否一遇到考试，就担心会考坏? ','你在工作或学习时，如果有人在注意你，你心里是否会紧张? ','你学习的时候，思想是否经常开小差? ']

In [45]:
final_filtered_df['stress_sum'] = final_filtered_df[stress_column].sum(axis = 1)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_164776\2876718585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_filtered_df['stress_sum'] = final_filtered_df[stress_column].sum(axis = 1)


In [46]:
final_filtered_df['stress_sum'].value_counts()

stress_sum
7.0    160
6.0    126
5.0     96
4.0     61
3.0     44
2.0     38
1.0     28
0.0     18
Name: count, dtype: int64

In [47]:
final_filtered_df['stress_label'] = final_filtered_df['stress_sum'].apply( lambda x: 1 if x > 6 else (0 if x < 5 else -1))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_164776\967548058.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_filtered_df['stress_label'] = final_filtered_df['stress_sum'].apply( lambda x: 1 if x > 6 else (0 if x < 5 else -1))


In [48]:

final_filtered_df.to_excel(f"./{outdir}/MiddleSchool_Depression_Anxiety.xlsx", index=False)

anxiety_df = final_filtered_df[final_filtered_df['anxiety_label'].isin([0, 1])]
anxiety_df = anxiety_df[['user_id'] + anxiety_columns + ['anxiety_sum']]